In [ ]:
import numpy as np
import math
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import csr_matrix

def read_file(filename):
    users = []
    news = []
    with open(filename, 'r') as f:
        data = f.readlines()
        for row in data:
            user, item = row.strip('\n').split('|')
            users.append(user)
            news.append(item)
    return users, news


def create_dictionary(users, news):
    users_dict = {}
    for user, item in zip(users, news):
        if user not in users_dict:
            users_dict[user] = list()
        users_dict[user].append(item)
    return users_dict

def make_user_interest_vector(user_interests):
    return [1 if interest in user_interests else 0 for interest in unique_interests]


users, news = read_file('viewed_news.csv')
users_interests = create_dictionary(users, news)
unique_interests = sorted(set(news))

index = 0
user_index = {}
matrix_of_interests = []
for user, value in users_interests.items():
    user_index[index]=user
    index += 1
    matrix_of_interests.append(value)
    


user_interest_matrix = map(make_user_interest_vector, matrix_of_interests)

user_similarity = cosine_similarity(csr_matrix(user_interest_matrix), dense_output=False)

In [ ]:
import scipy

def most_similar_users_to(user):
    pairs = []
    
    cx = scipy.sparse.coo_matrix(user_similarity[user,:])
    
    for user_id, similarity in zip(cx.col, cx.data):
        if user_id != user and similarity > 0:
            pairs.append((user_id, similarity))
            
    return sorted(pairs, key=lambda elem: elem[1], reverse=True)

# print(most_similar_users_to(1))

In [ ]:
def user_based_suggestions(user):
    suggestions = {}
    for user_id, similarity in most_similar_users_to(user):
        index = user_index[user_id]
        news = users_interests[index]
        for interest in news:
            if suggestions.has_key(interest):
                suggestions[interest] = suggestions.get(interest) + similarity
            else:
                suggestions[interest] = similarity
                
    suggestions = sorted(suggestions.items(), key=lambda x: x[1], reverse=True)
    final_suggestions = []
    index_user = user_index[user]
    
    for a in suggestions:
        if a not in users_interests[index_user]:
            final_suggestions.append(a)
    
    print(len(final_suggestions))
    return final_suggestions
user_based_suggestions(1)